In [1]:
import pandas as pd
import seaborn as sns

In [ ]:
metadata = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')

In [5]:
rnaseq = metadata[metadata['data_type'] == 'host_transcriptomics']

# display(rnaseq)

In [6]:
metagenomics = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metagenomics_metadata.csv')

In [ ]:
display(metagenomics)

In [8]:
# Filter df1 based on the Participant ID column in df2
metagenomics_filt = metagenomics[metagenomics['Participant ID'].isin(rnaseq['Participant ID'])]

# print(metagenomics_filt['Participant ID'].unique())

In [ ]:
rna_subset = rnaseq[['Participant ID', 'week_num']]
display(rna_subset)

In [10]:
rna_group = rna_subset.groupby('Participant ID').max()

In [11]:
rna_group = pd.DataFrame(rna_group)

In [ ]:
display(rna_group)

In [13]:
def find_closest_week(participant_id, week_num, metagenomic_df):
    # Filter the metagenomic_df for the same Participant ID
    participant_metagenomic = metagenomic_df[metagenomic_df["Participant ID"] == participant_id]
    
    # If no metagenomic samples for this participant, return None
    if participant_metagenomic.empty:
        return None
    
    # Calculate the difference between RNAseq week_num and metagenomic week_num
    participant_metagenomic["diff"] = participant_metagenomic["week_num"] - week_num

    print(participant_metagenomic['Participan ID', 'diff'])

    # # Select the sample with the smallest non-negative difference (closest week >= RNAseq week)
    # after_or_same = participant_metagenomic[participant_metagenomic["diff"] >= 0].sort_values(by="diff")
    # if not after_or_same.empty:
    #     return after_or_same.iloc[0]
    
    # # If no samples are found, select the sample with the smallest negative difference (closest week < RNAseq week)
    # before = participant_metagenomic[participant_metagenomic["diff"] < 0].sort_values(by="diff", ascending=False)
    # if not before.empty:
    #     return before.iloc[0]
    
    # # If no match is found, return None
    # return None


In [ ]:
merged_df = pd.merge(rnaseq, metagenomics_filt, on='Participant ID', how='left', suffixes=('_rnaseq', '_metagenomics'))

display(merged_df)

In [ ]:
viewing = merged_df[['Participant ID', 'week_num_rnaseq', 'week_num_metagenomics', 'Project_metagenomics']]

display(viewing)

In [ ]:
# Filter for the closest timepoint or earliest if none is before or at the RNA-seq timepoint
def find_closest_timepoint(group):
    # Filter for metagenomic samples that are before or at the RNA-seq timepoint
    valid_samples = group[group['week_num_metagenomics'] <= group['week_num_rnaseq'].iloc[0]]
    
    if not valid_samples.empty:
        # Find the sample with the closest (maximum) timepoint
        closest_sample = valid_samples.loc[valid_samples['week_num_metagenomics'].idxmax()]
    else:
        # If no valid sample exists, return the earliest metagenomic sample
        closest_sample = group.loc[group['week_num_metagenomics'].idxmin()]
    
    return pd.Series([closest_sample['External ID_metagenomics'], closest_sample['week_num_metagenomics']], 
                     index=['External ID_metagenomics', 'week_num_metagenomics'])

In [ ]:
closest_samples = merged_df.groupby(['Participant ID', 'Project_rnaseq']).apply(find_closest_timepoint).reset_index()

In [ ]:
rnaseq = rnaseq.rename(columns={'Project': 'Project_rnaseq'})

In [ ]:
# Merge the closest samples back to the original RNA-seq DataFrame
final_df = pd.merge(rnaseq, closest_samples, on=['Participant ID', 'Project_rnaseq'], how='left')

In [ ]:
final_df['External ID_metagenomics']

In [ ]:
final_df.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv', index=False)

In [ ]:
for_viewing = final_df[['Participant ID', 'External ID', 'External ID_metagenomics', 'week_num', 'week_num_metagenomics', 'Project_rnaseq']]

display(for_viewing)

In [ ]:
for_viewing.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv', index=False)

**Use Mapping File to Join MGX and RNA Data for Appropriate Samples**

In [29]:
# Load datasets
mgx = pd.read_csv('/Volumes/PGH-Backup/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered.tsv', sep='\t')
rna = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/tmm_normalized_counts.tsv', sep='\t')
mapping = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv')

In [30]:
# Tanspose so that Genes are columns and samples are rows in RNAseq data
rna = rna.set_index('Gene').T

rna.shape

(249, 20727)

In [31]:
display(mgx)
display(rna)
display(mapping)

,Unnamed: 0,DL-endopeptidase-A0A0B0HUJ2,DL-endopeptidase-A0A0P8W7Z2,DL-endopeptidase-A0A132I0H7,DL-endopeptidase-A0A174CGJ4,DL-endopeptidase-A0A174M4B0,DL-endopeptidase-A0A174TDS3,DL-endopeptidase-A0A1C5KMM8,DL-endopeptidase-A0A1C5KNH6,DL-endopeptidase-A0A1C5PVP3,...,Muramidase-T0UBX6,Muramidase-T0WIP7,Muramidase-U2ECY2,Muramidase-U7D9Q6,Muramidase-V1GVJ2,Muramidase-V5EDU1,Muramidase-W4UCM7,Muramidase-W5XD20,Muramidase-W9H589,sample_id
0,0,0.000000,0.000000e+00,8.383020e-07,0.000000,1.323079e-06,0.000000,0.000004,0.000003,0.000000e+00,...,0.0,0.0,0.000000,0.000145,0.0,0.0,0.0,0.0,0.0,CSM5FZ4M
1,1,0.000001,0.000000e+00,3.478880e-06,0.000002,4.373101e-06,0.000000,0.000018,0.000010,7.443690e-06,...,0.0,0.0,0.000000,0.000093,0.0,0.0,0.0,0.0,0.0,CSM5MCUO
2,2,0.000000,0.000000e+00,0.000000e+00,0.000000,9.575670e-07,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.000000,0.000146,0.0,0.0,0.0,0.0,0.0,CSM5MCVL
3,3,0.000000,0.000000e+00,1.302237e-06,0.000000,1.704135e-06,0.000000,0.000001,0.000000,0.000000e+00,...,0.0,0.0,0.000000,0.000143,0.0,0.0,0.0,0.0,0.0,CSM5MCVN
4,4,0.000000,0.000000e+00,2.120380e-06,0.000000,4.052097e-06,0.000000,0.000009,0.000008,0.000000e+00,...,0.0,0.0,0.000000,0.000130,0.0,0.0,0.0,0.0,0.0,CSM5MCW6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,1332,0.000000,0.000000e+00,2.761959e-06,0.000001,5.375752e-06,0.000003,0.000024,0.000004,0.000000e+00,...,0.0,0.0,0.000000,0.000117,0.0,0.0,0.0,0.0,0.0,PSMB4MBS
1333,1333,0.000000,3.981110e-07,2.111380e-06,0.000000,2.358866e-06,0.000000,0.000035,0.000004,9.643330e-07,...,0.0,0.0,0.000000,0.000126,0.0,0.0,0.0,0.0,0.0,PSMB4MC1
1334,1334,0.000000,0.000000e+00,9.060580e-07,0.000000,0.000000e+00,0.000000,0.000023,0.000000,2.045580e-06,...,0.0,0.0,0.000000,0.000146,0.0,0.0,0.0,0.0,0.0,PSMB4MC3
1335,1335,0.000000,0.000000e+00,0.000000e+00,0.000000,8.428045e-06,0.000004,0.000051,0.000006,1.476350e-05,...,0.0,0.0,0.000002,0.000083,0.0,0.0,0.0,0.0,0.0,PSMB4MC5


Gene,7SK,A1BG-AS1,A1CF,A2M,A2M-AS1,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZZZ3,hsa-mir-1199,hsa-mir-4537,hsa-mir-4538,hsa-mir-4539,hsa-mir-5195,hsa-mir-6080,hsa-mir-6723,hsa-mir-8072,snoU13
CSM5FZ1F,2.769182,1.129535,132.556390,152.487216,0.182183,2.076887,33.084442,55.893761,109.419142,0.947352,...,34.505470,7.396632,0.000000,0.036437,0.000000,0.036437,0.437239,165.968762,0.145746,1.894704
CSM5FZ1G,1.184610,0.704363,71.236670,286.771621,0.352181,0.992511,30.191542,55.708677,0.288148,2.817450,...,44.630975,13.863136,0.032016,0.064033,0.032016,0.064033,1.536791,135.237616,0.064033,1.600824
CSM5FZ1H,1.388848,0.413699,69.737894,149.700072,0.472799,0.768299,34.928047,62.202655,0.384149,3.871044,...,46.245680,18.320972,0.029550,0.118200,0.000000,0.177300,1.241098,142.608083,0.177300,1.832097
CSM5FZ1I,2.951572,1.855274,54.055930,804.963922,1.517951,3.513776,30.274694,53.100183,0.112441,3.148343,...,40.057047,15.010851,0.983857,0.393543,0.056220,0.140551,3.429445,127.901447,0.365433,1.686612
CSM5FZ1J,2.845846,0.856517,262.646720,839.800869,0.497332,3.647104,26.966463,37.189405,639.818020,1.713034,...,54.485518,10.913681,0.165777,0.221037,0.165777,0.000000,1.160442,139.943787,0.442073,3.647104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PSM7J4EW,3.959701,1.049321,44.328849,801.027642,0.871134,5.682170,31.677605,44.408043,1.504686,3.444940,...,41.378872,12.017691,1.108716,0.752343,0.494963,0.316776,3.385544,130.709718,0.653351,1.880858
PSM7J4EX,8.517420,1.860126,135.838157,721.802360,1.125866,7.391554,27.706089,29.419363,379.220975,2.055929,...,53.527577,10.304120,1.517471,0.905588,0.538458,1.174816,2.545436,118.509614,0.489507,2.839140
PSM7J4F3,3.990503,1.463184,105.748334,646.621132,0.611877,1.729218,28.997656,64.885581,6.996682,4.628984,...,53.180105,13.009040,0.239430,0.106413,0.159620,0.133017,1.968648,154.592092,0.079810,2.633732
PSM7J4F4,8.550605,5.651382,9.096835,1077.439196,1.428602,22.332414,29.874594,19.769334,5.000108,2.016850,...,49.433839,2.563080,2.836196,2.163912,1.113469,1.554655,1.869788,65.274517,0.651275,2.899222


,Participant ID,External ID,External ID_metagenomics,week_num,week_num_metagenomics,Project_rnaseq
0,C3002,CSM5FZ1G,CSM5FZ3T_P,0.0,0.0,C3002CSC1_TX
1,C3002,CSM5FZ1F,CSM5FZ3T_P,0.0,0.0,C3002CSC2_TX
2,C3002,CSM5FZ1I,CSM5FZ3T_P,0.0,0.0,C3002CSC3_TX
3,C3002,CSM5FZ1H,CSM5FZ3T_P,0.0,0.0,C3002CSC4_TX
4,C3003,CSM5FZ1J,CSM5FZ4E_P,1.0,2.0,C3003CSC1_TX
...,...,...,...,...,...,...
247,P6024,PSM7J4EU,PSM7J1BJ,2.0,2.0,P6024CSC2_TX
248,P6025,PSM7J4EX,PSM7J1BV,0.0,2.0,P6025CSC1_TX
249,P6025,PSM7J4EW,PSM7J1BV,0.0,2.0,P6025CSC2_TX
250,P6028,PSM7J4F4,PSM7J177,2.0,2.0,P6028CSC1_TX


In [32]:
rna = rna.reset_index().rename(columns={'index': 'sample_id'})
display(rna.head())

Gene,sample_id,7SK,A1BG-AS1,A1CF,A2M,A2M-AS1,A4GALT,AAAS,AACS,AADAC,...,ZZZ3,hsa-mir-1199,hsa-mir-4537,hsa-mir-4538,hsa-mir-4539,hsa-mir-5195,hsa-mir-6080,hsa-mir-6723,hsa-mir-8072,snoU13
0,CSM5FZ1F,2.769182,1.129535,132.556390,152.487216,0.182183,2.076887,33.084442,55.893761,109.419142,...,34.505470,7.396632,0.000000,0.036437,0.000000,0.036437,0.437239,165.968762,0.145746,1.894704
1,CSM5FZ1G,1.184610,0.704363,71.236670,286.771621,0.352181,0.992511,30.191542,55.708677,0.288148,...,44.630975,13.863136,0.032016,0.064033,0.032016,0.064033,1.536791,135.237616,0.064033,1.600824
2,CSM5FZ1H,1.388848,0.413699,69.737894,149.700072,0.472799,0.768299,34.928047,62.202655,0.384149,...,46.245680,18.320972,0.029550,0.118200,0.000000,0.177300,1.241098,142.608083,0.177300,1.832097
3,CSM5FZ1I,2.951572,1.855274,54.055930,804.963922,1.517951,3.513776,30.274694,53.100183,0.112441,...,40.057047,15.010851,0.983857,0.393543,0.056220,0.140551,3.429445,127.901447,0.365433,1.686612
4,CSM5FZ1J,2.845846,0.856517,262.646720,839.800869,0.497332,3.647104,26.966463,37.189405,639.818020,...,54.485518,10.913681,0.165777,0.221037,0.165777,0.000000,1.160442,139.943787,0.442073,3.647104


In [33]:
# Create list of enriched NOD2 genes
with open ('/Volumes/PGH-Backup/ibd_data/rnaseq/GSE22611_NOD2_VS_CTRL_TRANSDUCED_HEK293T_CELL_UP.v2024.1.Hs.grp', 'r') as f:
    gene_list = [line.strip() for line in f.readlines()]
    gene_list = gene_list[2:]

print(gene_list)

['ABCB1', 'ABHD17A', 'ACAD8', 'ACKR4', 'ACTG1', 'AGT', 'AIMP1', 'AK4', 'AMT', 'ANKRD27', 'APBB2', 'ARFGEF1', 'ARHGAP11A', 'ARHGAP6', 'ARHGEF11', 'ATG14', 'ATP13A3', 'ATP2B1', 'B3GNT2', 'BMERB1', 'BMS1', 'BNC2', 'BRD9', 'BSDC1', 'BTG3', 'C1GALT1C1', 'CAPG', 'CD40', 'CD48', 'CD68', 'CD99', 'CDC25B', 'CDK5RAP1', 'CEACAM6', 'CELA3B', 'CENPE', 'CHAF1B', 'CIR1', 'CLCC1', 'CLDN1', 'CNPY3', 'COLGALT1', 'COPS7A', 'CRYZ', 'CSK', 'CUL4A', 'CYB5B', 'DCTN3', 'DCXR', 'DDX46', 'DEGS1', 'DIXDC1', 'DOK2', 'DPAGT1', 'DRICH1', 'DUSP10', 'EBI3', 'EDAR', 'EGLN1', 'EIF2B2', 'EIF5', 'EIF5A', 'ENPP3', 'EXT2', 'FAM114A1', 'FAM193A', 'FAM32A', 'FASTKD5', 'FUT5', 'FXYD6', 'GALNT7', 'GCA', 'GDAP2', 'GLB1', 'GMFG', 'GNPDA1', 'GTF2E1', 'H1-1', 'HADHA', 'HECTD4', 'HIKESHI', 'HLA-DMA', 'HMGB1', 'HNRNPM', 'HTR3B', 'IFNA2', 'IGF1R', 'IGLV1-44', 'IL36G', 'IMPA1', 'INSIG1', 'ITPR1', 'KDM3B', 'KIF22', 'KIF3C', 'KMT5B', 'KPNA2', 'LARS2', 'LCMT1', 'LHX3', 'LILRA1', 'LIMD2', 'LZTR1', 'MAFF', 'MAN1A1', 'MARCHF5', 'MBD3', 'MBD

In [34]:
# Subset for genes present in table
present_genes = [gene for gene in gene_list if gene in rna.columns]

present_genes.append('sample_id')

In [35]:
# Subset rnaseq data
rna_filt = rna[present_genes]

display(rna_filt)

Gene,ABCB1,ABHD17A,ACAD8,ACKR4,ACTG1,AGT,AIMP1,AK4,AMT,ANKRD27,...,WTAP,XCL1,ZC2HC1A,ZDHHC3,ZNF185,ZNF282,ZNF287,ZNF675,ZNF813,sample_id
0,512.808859,15.995672,33.339499,82.674670,4299.702243,3.862281,107.050762,10.493744,21.679783,28.675613,...,111.896831,2.295506,1.056662,288.869449,16.287165,22.772882,2.113323,8.088928,3.680098,CSM5FZ1F
1,72.517330,7.972104,33.201091,21.387009,3537.917203,4.994571,89.934295,32.752860,25.869317,52.507029,...,99.283108,0.320165,7.747988,249.536453,34.513767,18.953757,2.881483,16.936718,5.859016,CSM5FZ1G
2,49.968974,10.637984,39.449190,23.580864,3506.693167,3.014095,95.357705,26.654059,29.431755,54.785617,...,94.264357,0.620549,7.210189,252.799866,58.952160,22.576166,4.077894,16.075176,8.599037,CSM5FZ1H
3,52.397428,21.841632,35.278311,13.914553,2924.529842,11.637626,70.865835,18.384076,20.492342,47.787354,...,83.627869,0.955747,7.139993,178.387381,16.781794,20.885885,3.401335,14.729749,6.830781,CSM5FZ1I
4,892.214169,14.339748,27.215129,28.292683,1772.216066,19.230183,64.929496,27.298018,25.170541,50.617378,...,79.158726,0.497332,4.420732,215.621187,5.691692,19.285442,1.934070,21.330030,10.499238,CSM5FZ1J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,29.222591,21.778354,29.638359,6.711693,2685.627364,16.254571,62.127703,25.559868,20.293466,42.606379,...,89.746615,1.504686,6.830484,147.795827,17.224698,21.560570,4.038895,14.670691,9.166707,PSM7J4EW
245,545.016952,16.643234,27.436860,12.139770,1930.468255,7.978962,52.377235,2.716763,24.181640,43.198982,...,82.995890,1.468521,6.045410,130.208828,9.741187,17.989378,3.402073,14.783108,9.741187,PSM7J4EX
246,167.840563,8.326850,32.296472,1.463184,2371.609235,1.649408,71.616230,69.434755,24.342069,49.455636,...,80.554957,1.303564,9.790034,178.668128,15.350135,14.844672,3.618056,23.969622,15.296929,PSM7J4F3
247,34.538560,29.643496,22.920662,10.735526,2001.156590,10.651490,49.391821,8.697667,9.937189,29.349372,...,137.187832,1.386585,9.769118,60.715595,12.374216,14.874270,5.420285,14.433084,9.937189,PSM7J4F4


In [ ]:
# # write out test subset rnaseq file:
# rna_filt.to_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq.tsv', index=False, sep='\t')

In [ ]:
merged_mgx = pd.merge(rna_filt, mapping, left_on='sample_id', right_on='External ID', how='inner')

display(merged_mgx)

In [26]:
merged_mgx['External ID_metagenomics'] = merged_mgx['External ID_metagenomics'].str.replace('_P', '')

In [ ]:
merged_mgx_rna = pd.merge(merged_mgx, mgx, left_on='External ID_metagenomics', right_on= 'sample_id', how='left')

display(merged_mgx_rna)

In [ ]:
subset_df = merged_mgx_rna[['sample_id_x', 'sample_id_y', 'External ID_metagenomics']]

display(subset_df)

In [ ]:
merged_mgx_rna.to_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq_mgx.tsv', index=False, sep='\t')

In [ ]:
view = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/test_subset_rnaseq_mgx.tsv', sep='\t')

display(view)